In [1]:
# ==============================================
# COMMON IMPORTS (Run this cell first)
# ==============================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, MinMaxScaler
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error
import seaborn as sns
import os
import zipfile
import requests
import io

# Set random seeds for reproducibility (optional)
np.random.seed(42)
tf.random.set_seed(42)

print("TensorFlow Version:", tf.__version__)
print("Keras Version:", keras.__version__)

/Users/aditya/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


TensorFlow Version: 2.19.0
Keras Version: 3.9.2


In [ ]:
# ==============================================
# Assignment 7: Transfer Learning (VGG16 on CIFAR-10)
# ==============================================
print("\n--- Assignment 7: Transfer Learning (VGG16 on CIFAR-10) ---")

# --- 1. Load and Prepare Dataset (CIFAR-10) ---
print("Loading CIFAR-10 dataset...")
(x_train_cifar_tl, y_train_cifar_tl), (x_test_cifar_tl, y_test_cifar_tl) = keras.datasets.cifar10.load_data()

# Preprocess for VGG16:
# - Resize images to VGG16 input size (e.g., 48x48 or larger - 224x224 is standard but slow)
# - Use VGG16 preprocessing function (scales pixels and converts RGB to BGR)
# - One-hot encode labels

target_size = (48, 48) # Smaller size for faster demo, VGG usually expects 224x224
num_classes_cifar_tl = 10

print(f"Resizing images to {target_size}...")
x_train_cifar_tl_resized = tf.image.resize(x_train_cifar_tl, target_size).numpy()
x_test_cifar_tl_resized = tf.image.resize(x_test_cifar_tl, target_size).numpy()

print("Applying VGG16 preprocessing...")
x_train_cifar_tl_preprocessed = keras.applications.vgg16.preprocess_input(x_train_cifar_tl_resized)
x_test_cifar_tl_preprocessed = keras.applications.vgg16.preprocess_input(x_test_cifar_tl_resized)

y_train_cifar_tl_onehot = keras.utils.to_categorical(y_train_cifar_tl, num_classes_cifar_tl)
y_test_cifar_tl_onehot = keras.utils.to_categorical(y_test_cifar_tl, num_classes_cifar_tl)

print(f"x_train shape after resize and preprocess: {x_train_cifar_tl_preprocessed.shape}")

# --- 2. Load Pretrained Model (VGG16) ---
print("Loading VGG16 base model (weights='imagenet')...")
base_model = keras.applications.VGG16(
    weights='imagenet',
    input_shape=target_size + (3,), # Use target_size
    include_top=False # Exclude the final classification layer
)

# Freeze the base model layers
base_model.trainable = False
print(f"Base model trainable: {base_model.trainable}")

# --- 3. Build Transfer Learning Model (Feature Extraction) ---
print("Building transfer learning model (Feature Extraction)...")
inputs = keras.Input(shape=target_size + (3,))
x = base_model(inputs, training=False) # Important: set training=False for frozen layers
x = layers.GlobalAveragePooling2D()(x) # Pool features
# x = layers.Flatten()(x) # Alternative pooling
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(num_classes_cifar_tl, activation='softmax')(x) # New classifier head
model_tl_feature_extraction = keras.Model(inputs, outputs, name="transfer_learning_FE")

model_tl_feature_extraction.summary()

# --- 4. Compile and Train (Feature Extraction Phase) ---
print("Compiling and training Feature Extraction model...")
# Use a lower learning rate for transfer learning typically
optimizer_tl_fe = keras.optimizers.Adam(learning_rate=0.0005)
model_tl_feature_extraction.compile(optimizer=optimizer_tl_fe,
                                     loss='categorical_crossentropy',
                                     metrics=['accuracy'])

epochs_tl_fe = 5 # Train only the head for a few epochs
history_tl_fe = model_tl_feature_extraction.fit(x_train_cifar_tl_preprocessed, y_train_cifar_tl_onehot,
                                                epochs=epochs_tl_fe,
                                                validation_split=0.1,
                                                batch_size=64)

# --- 5. Evaluate after Feature Extraction ---
print("Evaluating after Feature Extraction...")
loss_fe, accuracy_fe = model_tl_feature_extraction.evaluate(x_test_cifar_tl_preprocessed, y_test_cifar_tl_onehot, verbose=0)
print(f"Test Loss (FE): {loss_fe:.4f}")
print(f"Test Accuracy (FE): {accuracy_fe:.4f}")


# --- 6. Fine-Tuning (Optional) ---
print("\n--- Fine-Tuning Phase ---")
# Unfreeze some top layers of the base model
base_model.trainable = True

# Freeze layers up to a certain point (e.g., keep earlier blocks frozen)
# Fine-tune from 'block5_conv1' onwards
fine_tune_at = 'block5_conv1' # Example layer name
print(f"Unfreezing layers from {fine_tune_at} onwards...")

for layer in base_model.layers:
    if layer.name == fine_tune_at:
        break
    layer.trainable = False
    # print(f"Layer {layer.name} frozen.") # Uncomment to verify

print(f"Total layers: {len(base_model.layers)}, Trainable layers after unfreeze: {len(base_model.trainable_variables)}")

# Re-compile the model with a VERY low learning rate for fine-tuning
optimizer_tl_ft = keras.optimizers.Adam(learning_rate=1e-5) # e.g., 0.00001
model_tl_feature_extraction.compile(optimizer=optimizer_tl_ft,
                                     loss='categorical_crossentropy',
                                     metrics=['accuracy'])

print("Fine-tuning model...")
epochs_tl_ft = 5 # Train for a few more epochs
initial_epoch = epochs_tl_fe # Start counting epochs from where FE left off

history_tl_ft = model_tl_feature_extraction.fit(x_train_cifar_tl_preprocessed, y_train_cifar_tl_onehot,
                                                epochs=initial_epoch + epochs_tl_ft,
                                                initial_epoch=initial_epoch, # Important for history tracking
                                                validation_split=0.1,
                                                batch_size=64)


# --- 7. Evaluate after Fine-Tuning ---
print("Evaluating after Fine-Tuning...")
loss_ft, accuracy_ft = model_tl_feature_extraction.evaluate(x_test_cifar_tl_preprocessed, y_test_cifar_tl_onehot, verbose=0)
print(f"Test Loss (FT): {loss_ft:.4f}")
print(f"Test Accuracy (FT): {accuracy_ft:.4f}") # Should hopefully improve

# --- Plot combined training history (Optional) ---
# Combine histories for plotting
acc = history_tl_fe.history['accuracy'] + history_tl_ft.history['accuracy']
val_acc = history_tl_fe.history['val_accuracy'] + history_tl_ft.history['val_accuracy']
loss = history_tl_fe.history['loss'] + history_tl_ft.history['loss']
val_loss = history_tl_fe.history['val_loss'] + history_tl_ft.history['val_loss']

plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
plt.plot(acc, label='Train Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.plot([epochs_tl_fe-1, epochs_tl_fe-1], plt.ylim(), label='Start Fine Tuning', linestyle='--')
plt.title('Assignment 7: TL Accuracy (FE + FT)')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(loss, label='Train Loss')
plt.plot(val_loss, label='Validation Loss')
plt.plot([epochs_tl_fe-1, epochs_tl_fe-1], plt.ylim(), label='Start Fine Tuning', linestyle='--')
plt.title('Assignment 7: TL Loss (FE + FT)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()


--- Assignment 7: Transfer Learning (VGG16 on CIFAR-10) ---
Loading CIFAR-10 dataset...
Resizing images to (48, 48)...
Applying VGG16 preprocessing...
x_train shape after resize and preprocess: (50000, 48, 48, 3)
Loading VGG16 base model (weights='imagenet')...
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 15s 0us/step
Base model trainable: False
Building transfer learning model (Feature Extraction)...


Model: "transfer_learning_FE"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 48, 48, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg16 (Functional)              │ (None, 1, 1, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,848,586 (56.64 MB)

 Trainable params: 133,898 (523.04 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

Compiling and training Feature Extraction model...
Epoch 1/5
 38/704 ━━━━━━━━━━━━━━━━━━━━ 16:02 1s/step - accuracy: 0.1450 - loss: 17.8390